##### Notatki
Tutorial tutaj: https://towardsdatascience.com/image-captioning-with-keras-teaching-computers-to-describe-pictures-c88a46a311b8

Dane Flickr 8k:
https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_Dataset.zip

Polskie podpisy: http://zil.ipipan.waw.pl/Scwad/AIDe?action=AttachFile&do=get&target=AIDe_ANNOTATED_DESCRIPTIONS.zip

In [1]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import tensorflow as tf
from unidecode import unidecode
import csv
from math import ceil

Using TensorFlow backend.


In [2]:
learn_mode = 'en'

In [3]:
tokens_filename = "../../Flickr8k_text/Flickr8k.token.txt"
descriptions_filename = f'descriptions_{learn_mode}.txt'

test_images = []
train_images = []
    

In [4]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load descriptions
doc = load_doc(tokens_filename)
print(doc[:300])

1000268201_693b08cb0e.jpg#0	A child in a pink dress is climbing up a set of stairs in an entry way .
1000268201_693b08cb0e.jpg#1	A girl going into a wooden building .
1000268201_693b08cb0e.jpg#2	A little girl climbing into a wooden playhouse .
1000268201_693b08cb0e.jpg#3	A little girl climbing the s


In [5]:
def load_descriptions(doc):
	mapping = dict()
	# process lines
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		if len(line) < 2:
			continue
		# take the first token as the image id, the rest as the description
		image_id, image_desc = tokens[0], tokens[1:]
		# extract filename from image id
		image_id = image_id.split('.')[0]
		# convert description tokens back to string
		image_desc = ' '.join(image_desc)
		# create the list if needed
		if image_id not in mapping:
			mapping[image_id] = list()
		# store description
		mapping[image_id].append(image_desc)
	return mapping

# parse descriptions
descriptions = load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))

Loaded: 8092 


In [6]:
list(descriptions.keys())[:5]

['1000268201_693b08cb0e',
 '1001773457_577c3a7d70',
 '1002674143_1b742ab4b8',
 '1003163366_44323f5815',
 '1007129816_e794419615']

In [7]:
descriptions['1355703632_5683a4b6fb']

['A female softball player making a pitch .',
 'A girl is winding up to throw a softball .',
 'A teenage girl is catching a softball during a game .',
 'Kids playing softball .',
 'Young girl playing softball with white house in background']

In [8]:
descriptions['2521938802_853224f378']

['A cricket batsman swinging',
 'A man playing cricket',
 'A man wearing a baseball uniform swings back with a large bat .',
 'A man wearing white clothes playing cricket .',
 'The baseball player holds up his bat ready to swing .']

In [9]:
def clean_descriptions(descriptions):
	# prepare translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)
	for key, desc_list in descriptions.items():
		for i in range(len(desc_list)):
			desc = desc_list[i]
			# tokenize
			desc = desc.split()
			# convert to lower case
			desc = [word.lower() for word in desc]
			# remove punctuation from each token
			desc = [w.translate(table) for w in desc]
			# remove hanging 's' and 'a'
			desc = [word for word in desc if len(word)>1]
			# remove tokens with numbers in them
			desc = [word for word in desc if word.isalpha()]
			# store as string
			desc_list[i] =  ' '.join(desc)

# clean descriptions
clean_descriptions(descriptions)

In [10]:
descriptions['1355703632_5683a4b6fb']

['female softball player making pitch',
 'girl is winding up to throw softball',
 'teenage girl is catching softball during game',
 'kids playing softball',
 'young girl playing softball with white house in background']

In [11]:
descriptions['2521938802_853224f378']

['cricket batsman swinging',
 'man playing cricket',
 'man wearing baseball uniform swings back with large bat',
 'man wearing white clothes playing cricket',
 'the baseball player holds up his bat ready to swing']

In [12]:
# convert the loaded descriptions into a vocabulary of words
def to_vocabulary(descriptions):
	# build a list of all description strings
	all_desc = set()
	for key in descriptions.keys():
		[all_desc.update(d.split()) for d in descriptions[key]]
	return all_desc

# summarize vocabulary
vocabulary = to_vocabulary(descriptions)
print('Original Vocabulary Size: %d' % len(vocabulary))

Original Vocabulary Size: 8763


In [13]:
# save descriptions to file, one per line
def save_descriptions(descriptions, filename):
	lines = list()
	for key, desc_list in descriptions.items():
		for desc in desc_list:
			lines.append(key + ' ' + desc)
	data = '\n'.join(lines)
	file = open(filename, 'w', encoding='utf-8')
	file.write(data)
	file.close()

save_descriptions(descriptions, descriptions_filename)

In [14]:
# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)

train = []

# load training dataset (6K)
for i in range(0,5): 
    train.append(load_set(f'randomState/trainImages{i}.txt'))
    print('Dataset: %d' % len(train[i]))

Dataset: 700
Dataset: 700
Dataset: 700
Dataset: 700
Dataset: 700


In [15]:
# Below path contains all the images
images = '../../Flickr8k_Dataset/'
# Create a list of all image names in the directory
img = glob.glob(images + '*.jpg')

In [16]:
train_images = []
train_img = []

# Read the train image names in a set
for i in range(0,5):
    train_images.append(set(open(f'randomState/trainImages{i}.txt', 'r').read().strip().split('\n')))
    # Create a list of all the training images with their full path names
    train_img.append([])

    for im in img: # img is list of full path names of all images
        if im[len(images):] in train_images[i]: # Check if the image belongs to training set
            train_img[i].append(im) # Add it to the list of train images

In [17]:
test_images = []
test_img = []

# Read the train image names in a set
for i in range(0,5):
    test_images.append(set(open(f'randomState/testImages{i}.txt', 'r').read().strip().split('\n')))

    # Create a list of all the training images with their full path names
    test_img.append([])

    for im in img: # img is list of full path names of all images
        if im[len(images):] in test_images[i]: # Check if the image belongs to training set
            test_img[i].append(im) # Add it to the list of train images

In [18]:
# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions

train_descriptions = []

# descriptions
for i in range(0,5):
    train_descriptions.append(load_clean_descriptions(descriptions_filename, train[i]))
    print('Descriptions: train=%d' % len(train_descriptions[i]))

Descriptions: train=700
Descriptions: train=700
Descriptions: train=700
Descriptions: train=700
Descriptions: train=700


In [19]:
def preprocess(image_path):
    # Convert all the images to size 299x299 as expected by the inception v3 model
    img = image.load_img(image_path, target_size=(299, 299))
    # Convert PIL image to numpy array of 3-dimensions
    x = image.img_to_array(img)
    # Add one more dimension
    x = np.expand_dims(x, axis=0)
    # preprocess the images using preprocess_input() from inception module
    x = preprocess_input(x)
    return x

In [20]:
# Load the inception v3 model
model = InceptionV3(weights='imagenet')

In [21]:
# Create a new model, by removing the last layer (output layer) from the inception v3
model_new = Model(model.input, model.layers[-2].output)

In [22]:
# Function to encode a given image into a vector of size (2048, )
def encode(image):
    image = preprocess(image) # preprocess the image
    fea_vec = model_new.predict(image) # Get the encoding vector for the image
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1]) # reshape from (1, 2048) to (2048, )
    return fea_vec

###### Notatki
Poniższy kod zajmuje bardzo dużo czasu.

In [23]:
# Call the funtion to encode all the train images
# This will take a while on CPU - Execute this only once
encoding_train = []

for i in range(0,5):
    start = time()
    encoding_train.append({})
    for img in train_img[i]:
        encoding_train[i][img[len(images):]] = encode(img)
        print('.', end='')
    print("Time taken in seconds =", time()-start)

............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................Time taken in seconds = 203.4167582988739
..................................................................................................................................................................................................................................................................

In [24]:
import os
import pickle


for i in range(0,5):
    writepath = f'Pickle/encoded_train_images_{learn_mode}_{i}.pkl'

    mode = 'a' if os.path.exists(writepath) else 'wb'

    # Save the bottleneck train features to disk
    with open(writepath, "wb") as encoded_pickle:
        pickle.dump(encoding_train[i], encoded_pickle)

In [25]:
# Call the funtion to encode all the test images - Execute this only once
encoding_test = []

for i in range(0,5):
    start = time()
    encoding_test.append({})
    for img in test_img[i]:
        encoding_test[i][img[len(images):]] = encode(img)
        print('.', end='')
    print("Time taken in seconds =", time()-start)

............................................................................................................................................................................................................................................................................................................Time taken in seconds = 75.66325879096985
............................................................................................................................................................................................................................................................................................................Time taken in seconds = 76.40600776672363
............................................................................................................................................................................................................................................................................................................Time taken in se

In [26]:
# Save the bottleneck test features to disk
for i in range(0,5):
    with open(f'Pickle/encoded_test_images_{learn_mode}_{i}.pkl', "wb") as encoded_pickle:
        pickle.dump(encoding_test[i], encoded_pickle)

In [27]:
train_features = []

for i in range(0,5):
    train_features.append(load(open(f'Pickle/encoded_train_images_{learn_mode}_{i}.pkl', "rb")))
    print('Photos: train=%d' % len(train_features[i]))

Photos: train=700
Photos: train=700
Photos: train=700
Photos: train=700
Photos: train=700


In [28]:
# Create a list of all the training captions
all_train_captions = []

for i in range(0,5):
    all_train_captions.append([])
    for key, val in train_descriptions[i].items():
        for cap in val:
            all_train_captions[i].append(cap)
    print(len(all_train_captions[i]))

3500
3500
3500
3500
3500


In [29]:
# Consider only words which occur at least 10 times in the corpus
word_count_threshold = 10
word_counts = []
vocab = []

for i in range(0,5):
    word_counts.append({})
    nsents = 0
    for sent in all_train_captions[i]:
        nsents += 1
        for w in sent.split(' '):
            word_counts[i][w] = word_counts[i].get(w, 0) + 1

    vocab.append([w for w in word_counts[i] if word_counts[i][w] >= word_count_threshold])
    print('preprocessed words %d -> %d' % (len(word_counts[i]), len(vocab[i])))

preprocessed words 2802 -> 438
preprocessed words 2767 -> 434
preprocessed words 2788 -> 453
preprocessed words 2720 -> 448
preprocessed words 2786 -> 444


In [30]:
ixtoword = []
wordtoix = []

for i in range(0,5):
    ixtoword.append({})
    wordtoix.append({})

    ix = 1
    for w in vocab[i]:
        wordtoix[i][w] = ix
        ixtoword[i][ix] = w
        ix += 1

In [31]:
vocab_size = []

for i in range(0,5):
    vocab_size.append(len(ixtoword[i]) + 1) # one for appended 0's

vocab_size

[439, 435, 454, 449, 445]

In [32]:
# convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# calculate the length of the description with the most words
def get_max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

max_length = []

# determine the maximum sequence length
for i in range(0,5):
    max_length.append(get_max_length(train_descriptions[i]))
    print('Description Length: %d' % max_length[i])

Description Length: 30
Description Length: 30
Description Length: 30
Description Length: 30
Description Length: 30


In [33]:
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch, vocab_size):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            # retrieve the photo feature
            photo = photos[key+'.jpg']
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n==num_photos_per_batch:
#                 yield [[array(X1), array(X2)], array(y)]
                yield ([array(X1), array(X2)], array(y)) 
                X1, X2, y = list(), list(), list()
                n=0

Embedings pobrane stąd: https://github.com/sdadas/polish-nlp-resources/releases/download/v1.0/glove.zip

In [34]:
# Load Glove vectors
glove_dir = '../../glove'
embeddings_index = []

for i in range(0,5):
    embeddings_index.append({}) # empty dictionary

f = open(os.path.join(glove_dir, 'glove.6B.200d.txt'), encoding="utf-8")

for line in f:
    values = line.split()
    coefs = np.asarray(values[1:], dtype='float32')
    for i in range(0,5):
        word = values[0]
        embeddings_index[i][word] = coefs


In [35]:
embedding_dim = 200
embedding_matrix = []

# Get 200-dim dense vector for each of the 10000 words in out vocabulary
for i in range(0,5):
    embedding_matrix.append(np.zeros((vocab_size[i], embedding_dim)))

    for word, j in wordtoix[i].items():
        #if i < max_words:
        embedding_vector = embeddings_index[i].get(word)
        if embedding_vector is not None:
            # Words not found in the embedding index will be all zeros
            embedding_matrix[i][j] = embedding_vector

In [36]:
model = []

for i in range(0,5):
    inputs1 = Input(shape=(2048,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)
    inputs2 = Input(shape=(max_length[i],))
    se1 = Embedding(vocab_size[i], embedding_dim, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)
    decoder1 = add([fe2, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size[i], activation='softmax')(decoder2)
    model.append(Model(inputs=[inputs1, inputs2], outputs=outputs))

In [37]:
for i in range(0,5):
    model[i].layers[2].set_weights([embedding_matrix[i]])
    model[i].layers[2].trainable = False

In [38]:
for i in range(0,5):
    model[i].compile(loss='categorical_crossentropy', optimizer='adam')

In [39]:
epochs = 10
number_pics_per_bath = 3
steps = []
for i in range(0,5):
    steps.append(len(train_descriptions[i])//number_pics_per_bath)

Kolejny dość czasochłonny krok

In [40]:
for i in range(0,5):
    for epoch in range(epochs):
        generator = data_generator(train_descriptions[i], train_features[i], wordtoix[i], max_length[i], number_pics_per_bath, vocab_size[i])
        print(generator)
        model[i].fit_generator(generator, epochs=1, steps_per_epoch=steps[i], verbose=1)
        writepath = f'model_weights/model_{str(epoch)}_{learn_mode}_{i}.h5'

        mode = 'a' if os.path.exists(writepath) else 'wb'
    #     file = open(writepath, mode)
        model[i].save(writepath)

<generator object data_generator at 0x0000029013DBF5C8>
Epoch 1/1
233/233 [==============================] - 78s 333ms/step - loss: 4.7009
<generator object data_generator at 0x0000029013DBF448>
Epoch 1/1
233/233 [==============================] - 74s 318ms/step - loss: 4.0507
<generator object data_generator at 0x0000029013DBF5C8>
Epoch 1/1
233/233 [==============================] - 72s 308ms/step - loss: 3.6733
<generator object data_generator at 0x000002901732FD48>
Epoch 1/1
233/233 [==============================] - 71s 303ms/step - loss: 3.3998
<generator object data_generator at 0x0000029013DBF448>
Epoch 1/1
233/233 [==============================] - 70s 299ms/step - loss: 3.1983
<generator object data_generator at 0x000002901732FD48>
Epoch 1/1
233/233 [==============================] - 67s 289ms/step - loss: 3.0406
<generator object data_generator at 0x0000029013DBF5C8>
Epoch 1/1
233/233 [==============================] - 67s 288ms/step - loss: 2.9212
<generator object data_gene

In [41]:
for i in range(0,5):
    for epoch in range(epochs):
        generator = data_generator(train_descriptions[i], train_features[i], wordtoix[i], max_length[i], number_pics_per_bath, vocab_size[i])
        model[i].fit_generator(generator, epochs=1, steps_per_epoch=steps[i], verbose=1)
        model[i].save(f'model_weights/model_{str(epoch)}_{learn_mode}_{i}.h5')

Epoch 1/1
233/233 [==============================] - 68s 290ms/step - loss: 2.4894
Epoch 1/1
233/233 [==============================] - 69s 297ms/step - loss: 2.3915
Epoch 1/1
233/233 [==============================] - 69s 294ms/step - loss: 2.3094
Epoch 1/1
233/233 [==============================] - 68s 291ms/step - loss: 2.2296
Epoch 1/1
233/233 [==============================] - 67s 289ms/step - loss: 2.1482
Epoch 1/1
233/233 [==============================] - 68s 292ms/step - loss: 2.0602
Epoch 1/1
233/233 [==============================] - 67s 288ms/step - loss: 1.9919
Epoch 1/1
233/233 [==============================] - 69s 296ms/step - loss: 1.9246
Epoch 1/1
233/233 [==============================] - 70s 300ms/step - loss: 1.8724
Epoch 1/1
233/233 [==============================] - 67s 288ms/step - loss: 1.8183
Epoch 1/1
233/233 [==============================] - 67s 289ms/step - loss: 2.4740
Epoch 1/1
233/233 [==============================] - 68s 292ms/step - loss: 2.3854
Epoc

In [42]:
# model.optimizer.lr = 0.0001
epochs = 10
number_pics_per_bath = 6
steps = []
for i in range(0,5):
    steps.append(len(train_descriptions[i])//number_pics_per_bath)

In [43]:
for i in range(0,5):
    for epoch in range(epochs):
        generator = data_generator(train_descriptions[i], train_features[i], wordtoix[i], max_length[i], number_pics_per_bath, vocab_size[i])
        model[i].fit_generator(generator, epochs=1, steps_per_epoch=steps[i], verbose=1)

Epoch 1/1
116/116 [==============================] - 64s 555ms/step - loss: 1.8764
Epoch 1/1
116/116 [==============================] - 63s 544ms/step - loss: 1.8161
Epoch 1/1
116/116 [==============================] - 63s 541ms/step - loss: 1.7459
Epoch 1/1
116/116 [==============================] - 64s 548ms/step - loss: 1.6944
Epoch 1/1
116/116 [==============================] - 64s 548ms/step - loss: 1.6613
Epoch 1/1
116/116 [==============================] - 64s 555ms/step - loss: 1.6111
Epoch 1/1
116/116 [==============================] - 65s 561ms/step - loss: 1.5692
Epoch 1/1
116/116 [==============================] - 65s 564ms/step - loss: 1.5263
Epoch 1/1
116/116 [==============================] - 64s 554ms/step - loss: 1.5056
Epoch 1/1
116/116 [==============================] - 63s 543ms/step - loss: 1.4812
Epoch 1/1
116/116 [==============================] - 63s 540ms/step - loss: 1.9181
Epoch 1/1
116/116 [==============================] - 63s 542ms/step - loss: 1.8698
Epoc

In [44]:
for i in range(0,5):
    model[i].save_weights(f'model_weights/model_30_{learn_mode}_{i}.h5')

In [45]:
for i in range(0,5):
    model[i].load_weights(f'model_weights/model_30_{learn_mode}_{i}.h5')